## For this lab me and my friends have put together our voices into one dataset and build a LSTM model to recognize which speaker the voice corresponds to. The results are quite amazing though the dataset is small. LSTM does a brilliant job when it comes to sequential or series data.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install keras==2.0.3

     |████████████████████████████████| 196 kB 9.8 MB/s 
     |████████████████████████████████| 2.8 MB 40.3 MB/s 
  Created wheel for keras: filename=Keras-2.0.3-py3-none-any.whl size=232959 sha256=1f283cbbe0de8f99507fd423edc353ac07acfbb9b5bcb9699baaf45d22dfb658
  Stored in directory: /root/.cache/pip/wheels/7d/40/7d/6b95f24cb13373df91040f04dd0c9e4e70a654c3c7aa6970e4
  Created wheel for theano: filename=Theano-1.0.5-py3-none-any.whl size=2668111 sha256=a897364604a1aca92e9c0941965e6bcbf409752fc1e47b92e3f7152125cf035b
  Stored in directory: /root/.cache/pip/wheels/26/68/6f/745330367ce7822fe0cd863712858151f5723a0a5e322cc144
Successfully built keras theano
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7.

In [3]:
!pip install tensorflow==1.13.2

     |████████████████████████████████| 92.7 MB 1.3 MB/s 
     |████████████████████████████████| 3.2 MB 45.1 MB/s 
     |████████████████████████████████| 367 kB 61.0 MB/s 
     |████████████████████████████████| 50 kB 7.2 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.7.0
    Uninstalling tensorflow-2.7.0:
      Successfully uninstalled tensorflow-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.6 requires tensorflow>=2.0.0, but you

In [4]:
import glob
import numpy as np
import random
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

import keras
from keras.layers import LSTM, Dense, Dropout, Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

## **LOAD DATASET**

The dataset is collected with the voices of 4 people. Under each person we have 20 voice records. 

In [6]:
!unzip '/content/drive/MyDrive/DeepLearning/Ownvoice.zip' -d '/content/drive/MyDrive/DeepLearning/'

Archive:  /content/drive/MyDrive/DeepLearning/Ownvoice.zip
   creating: /content/drive/MyDrive/DeepLearning/Ownvoice/
  inflating: /content/drive/MyDrive/DeepLearning/Ownvoice/1_Dhaxina_10.wav  
  inflating: /content/drive/MyDrive/DeepLearning/__MACOSX/Ownvoice/._1_Dhaxina_10.wav  
  inflating: /content/drive/MyDrive/DeepLearning/Ownvoice/1_Kavin_4.wav  
  inflating: /content/drive/MyDrive/DeepLearning/__MACOSX/Ownvoice/._1_Kavin_4.wav  
  inflating: /content/drive/MyDrive/DeepLearning/Ownvoice/0_Dhaxina_4.wav  
  inflating: /content/drive/MyDrive/DeepLearning/__MACOSX/Ownvoice/._0_Dhaxina_4.wav  
  inflating: /content/drive/MyDrive/DeepLearning/Ownvoice/1_Bharath_0.wav  
  inflating: /content/drive/MyDrive/DeepLearning/__MACOSX/Ownvoice/._1_Bharath_0.wav  
  inflating: /content/drive/MyDrive/DeepLearning/Ownvoice/0_Dhaxina_5.wav  
  inflating: /content/drive/MyDrive/DeepLearning/__MACOSX/Ownvoice/._0_Dhaxina_5.wav  
  inflating: /content/drive/MyDrive/DeepLearning/Ownvoice/1_Bharath_1

In [7]:
#SEED = 2017
DATA_DIR = '/content/drive/MyDrive/DeepLearning/Ownvoice/' 

In [14]:
files = glob.glob(DATA_DIR + "*.wav")
#print(files)
X_train, X_val = train_test_split(files, test_size=0.2)

print('# Training examples: {}'.format(len(X_train)))
print('# Validation examples: {}'.format(len(X_val)))

# Training examples: 64
# Validation examples: 16


In [15]:
labels = []
for i in range(len(X_train)):
    label = X_train[i].split('/')[-1].split('_')[1]
    if label not in labels:
        labels.append(label)
print(labels)

['Bharath', 'Kavin', 'Karthik', 'Dhaxina']


In [16]:
label_binarizer = LabelBinarizer()
label_binarizer.fit(list(set(labels)))

def one_hot_encode(x): return label_binarizer.transform(x)

In [31]:
n_features = 20
max_length = 800
n_classes = len(labels)

In [34]:
def batch_generator(data, batch_size=16):
    while 1:
        random.shuffle(data)
        X, y = [], []
        for i in range(batch_size):
            wav = data[i]
            wave, sr = librosa.load(wav, mono=True)
            label = wav.split('/')[-1].split('_')[1]
            y.append(label)
            mfcc = librosa.feature.mfcc(wave, sr)
            mfcc = np.pad(mfcc, ((0,0), (0, max_length-len(mfcc[0]))), mode='constant', constant_values=0) 
            X.append(np.array(mfcc))
        yield np.array(X), np.array(one_hot_encode(y))

In [35]:
learning_rate = 0.001
batch_size = 64
n_epochs = 50
dropout = 0.5

input_shape = (n_features, max_length)
steps_per_epoch = 50

## **BUILD MODEL**

In [36]:
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=input_shape,
dropout=dropout))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(n_classes, activation='softmax'))

In [37]:
opt = Adam(lr=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 20, 256)           1082368   
_________________________________________________________________
flatten_2 (Flatten)          (None, 5120)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               655488    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 516       
Total params: 1,738,372
Trainable params: 1,738,372
Non-trainable params: 0
_________________________________________________________________


In [38]:
callbacks = [ModelCheckpoint('/content/drive/MyDrive/DeepLearning/Speaker_Recognition/checkpoint/own_voice_recognition_best_model_{epoch:02d}.hdf5', save_best_only=True),
            EarlyStopping(monitor='val_acc', patience=2)]

## **TRAINING**

In [39]:
history = model.fit_generator(
 generator=batch_generator(X_train, 64),
 steps_per_epoch=steps_per_epoch,
 epochs=n_epochs,
 verbose=1,
 validation_data=batch_generator(X_val, 16),
 validation_steps=5,
 callbacks=callbacks
 )

Epoch 1/50
50/50 [==============================] - 416s - loss: 0.1440 - acc: 0.9494 - val_loss: 9.5128e-04 - val_acc: 1.0000
Epoch 2/50
50/50 [==============================] - 400s - loss: 0.0018 - acc: 0.9994 - val_loss: 0.0132 - val_acc: 1.0000
Epoch 3/50
50/50 [==============================] - 401s - loss: 0.0012 - acc: 0.9997 - val_loss: 0.0084 - val_acc: 1.0000
Epoch 4/50
50/50 [==============================] - 401s - loss: 6.8978e-04 - acc: 1.0000 - val_loss: 1.0106e-04 - val_acc: 1.0000


## **TESTING**

In [40]:
DATA_DIR = '/content/drive/MyDrive/DeepLearning/KarthikVoiceTest/' 

In [47]:
test = glob.glob(DATA_DIR + "*.wav")

In [48]:
X_test, y_test = [], []
for i in range(2):
    wav = test[i]
    wave, sr = librosa.load(wav, mono=True)
    label = wav.split('/')[-1].split('_')[1]
    y_test.append(label)
    mfcc = librosa.feature.mfcc(wave, sr)
    mfcc = np.pad(mfcc, ((0,0), (0, max_length-len(mfcc[0]))), mode='constant', constant_values=0) 
    X_test.append(np.array(mfcc))

In [45]:
X_test

[array([[-809.47577, -809.47577, -809.47577, ...,    0.     ,    0.     ,
            0.     ],
        [   0.     ,    0.     ,    0.     , ...,    0.     ,    0.     ,
            0.     ],
        [   0.     ,    0.     ,    0.     , ...,    0.     ,    0.     ,
            0.     ],
        ...,
        [   0.     ,    0.     ,    0.     , ...,    0.     ,    0.     ,
            0.     ],
        [   0.     ,    0.     ,    0.     , ...,    0.     ,    0.     ,
            0.     ],
        [   0.     ,    0.     ,    0.     , ...,    0.     ,    0.     ,
            0.     ]], dtype=float32),
 array([[-782.11017, -782.11017, -782.11017, ...,    0.     ,    0.     ,
            0.     ],
        [   0.     ,    0.     ,    0.     , ...,    0.     ,    0.     ,
            0.     ],
        [   0.     ,    0.     ,    0.     , ...,    0.     ,    0.     ,
            0.     ],
        ...,
        [   0.     ,    0.     ,    0.     , ...,    0.     ,    0.     ,
            0.     

In [49]:
y_test

['Karthik', 'Karthik']

In [53]:
X_test = np.array(X_test)

In [54]:
y_pred = model.predict(X_test, verbose=1)

2/2 [==============================] - 0s


In [56]:
print(labels)

['Bharath', 'Kavin', 'Karthik', 'Dhaxina']


In [55]:
y_pred

array([[2.2242505e-10, 1.8034162e-09, 1.0000000e+00, 2.0035082e-09],
       [2.0555129e-12, 3.6633155e-11, 1.0000000e+00, 4.7126265e-11]],
      dtype=float32)

## As we can see the 3rd element in array has probability of 1 suggesting that the speaker is Karthik.